*ETL*

Cargar todas las librerías a utilizar:

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import json
import ast
import pyarrow
import uvicorn
import os
import pyarrow

Importar el CSV a Visual Code a través de Python de manera adecuada para que funcione en distintos entornos

In [4]:
base_dir = os.getcwd()
file_path = os.path.join(base_dir, 'C:\\Users\\jessi\\Desktop\\Henry\\PROYECTO FINAL INDIVIDUAL\\movies_dataset.csv')
df = pd.read_csv(file_path)


C:\Users\jessi\AppData\Local\Temp\ipykernel_20820\1018976034.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Revisar el tipo de dato en cada columna del DataFrame para seleccionar cuáles son relevantes para el proyecto

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Se pasa a int la columna budget y la columna ID que servirán para las futuras funciones.

In [6]:
# Reemplazar el valor '1997-08-20' con 0
df['budget'] = df['budget'].replace(pd.Timestamp('1997-08-20'), 0)

# Convertir la columna 'budget' a tipo int
df['budget'] = df['budget'].astype(int, errors='ignore')




In [7]:
# Reemplazar el valor '1997-08-20' con 0
df['id'] = df['id'].replace(pd.Timestamp('1997-08-20'), 0)

# Reemplazar NaT con 0
df['id'] = df['id'].fillna(0)

# Convertir la columna 'id' a tipo int
df['id'] = df['id'].astype(int, errors='ignore')

Eliminar video,imdb_id,adult,original_title,poster_path y homepage.


In [8]:
df=df.drop(columns=["video","imdb_id","adult","original_title","poster_path","homepage"])

Revisamos si eliminamos las columnas correctamente

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45466 non-null  object 
 2   genres                 45466 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   production_companies   45463 non-null  object 
 8   production_countries   45463 non-null  object 
 9   release_date           45379 non-null  object 
 10  revenue                45460 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20412 non-null  object 
 15  ti

Identificar las variables que tienen datos anidados para darles el correcto tratamiento:
- belongs_to_collection
- genres
- production_companies
- production_coutries
- spoken_languages


In [10]:
def extract_full_collection_info(row):
    if pd.isna(row['belongs_to_collection']):
        return pd.Series([None, None, None, None], 
                         index=['collection_id', 'collection_name', 'collection_poster_path', 'collection_backdrop_path'])
    else:
        try:
            collection = json.loads(row['belongs_to_collection'].replace("'", "\""))
            return pd.Series([collection.get('id'), collection.get('name'), collection.get('poster_path'), collection.get('backdrop_path')], 
                             index=['collection_id', 'collection_name', 'collection_poster_path', 'collection_backdrop_path'])
        except:
            return pd.Series([None, None, None, None], 
                             index=['collection_id', 'collection_name', 'collection_poster_path', 'collection_backdrop_path'])

# Aplicar la función a cada fila del DataFrame
full_collection_info = df.apply(extract_full_collection_info, axis=1)

# Añadir las nuevas columnas al DataFrame original
aux = pd.concat([df, full_collection_info],axis=1)

In [34]:

def genres_ids(companies):
    try:
        companies_list = ast.literal_eval(companies)
        return [company['id'] for company in companies_list]
    except (ValueError, SyntaxError, TypeError):
        return []

def genres_names(companies):
    try:
        companies_list = ast.literal_eval(companies)
        return [company['name'] for company in companies_list]
    except (ValueError, SyntaxError, TypeError):
        return []

df['genres_ids'] = df['genres'].apply(genres_ids)
df['genres_names'] = df['genres'].apply(genres_names)

df['genres_names_array'] = df['genres_names'].apply(lambda x: x if isinstance(x,list)else[])
aux1 = df[['genres_names_array']]                                                                              


In [12]:
aux1

,genres_names_array
0,"[Animation, Comedy, Family]"
1,"[Adventure, Fantasy, Family]"
2,"[Romance, Comedy]"
3,"[Comedy, Drama, Romance]"
4,[Comedy]
...,...
45461,"[Drama, Family]"
45462,[Drama]
45463,"[Action, Drama, Thriller]"
45464,[]


In [35]:

def extract_production_company_ids(companies):
    try:
        companies_list = ast.literal_eval(companies)
        return [company['id'] for company in companies_list]
    except (ValueError, SyntaxError, TypeError):
        return []

def extract_production_company_names(companies):
    try:
        companies_list = ast.literal_eval(companies)
        return [company['name'] for company in companies_list]
    except (ValueError, SyntaxError, TypeError):
        return []

df['production_company_ids'] = df['production_companies'].apply(extract_production_company_ids)
df['production_company_names'] = df['production_companies'].apply(extract_production_company_names)

df['company_names_array'] = df['production_company_names'].apply(lambda x: x if isinstance(x,list)else[])
aux2 = df[['company_names_array']]                                                                              




In [14]:
aux2

,company_names_array
0,[Pixar Animation Studios]
1,"[TriStar Pictures, Teitler Film, Interscope Co..."
2,"[Warner Bros., Lancaster Gate]"
3,[Twentieth Century Fox Film Corporation]
4,"[Sandollar Productions, Touchstone Pictures]"
...,...
45461,[]
45462,[Sine Olivia]
45463,[American World Pictures]
45464,[Yermoliev]


In [36]:

def extract_production_company_iso(companies):
    try:
        companies_list = ast.literal_eval(companies)
        return [company['iso_3166_1'] for company in companies_list]
    except (ValueError, SyntaxError, TypeError):
        return []

def extract_production_company_paises(companies):
    try:
        companies_list = ast.literal_eval(companies)
        return [company['name'] for company in companies_list]
    except (ValueError, SyntaxError, TypeError):
        return []

df['production_countries_iso'] = df['production_countries'].apply(extract_production_company_iso)
df['production_countries_names'] = df['production_countries'].apply(extract_production_company_paises)

df['countries_names_array'] = df['production_countries_names'].apply(lambda x: x if isinstance(x,list)else[])
aux3 = df[['countries_names_array']]    



In [16]:
aux3

,countries_names_array
0,[United States of America]
1,[United States of America]
2,[United States of America]
3,[United States of America]
4,[United States of America]
...,...
45461,[Iran]
45462,[Philippines]
45463,[United States of America]
45464,[Russia]


In [37]:
def extract_iso_languages(languages):
    try:
        languages_list = ast.literal_eval(languages)
        return [lang['iso_639_1'] for lang in languages_list]
    except (ValueError, SyntaxError, TypeError):
        return []

def extract_language_names(languages):
    try:
        languages_list = ast.literal_eval(languages)
        return [lang['name'] for lang in languages_list]
    except (ValueError, SyntaxError, TypeError):
        return []


# Aplicar las funciones a la columna 'spoken_languages'
df['spoken_languages_iso'] = df['spoken_languages'].apply(extract_iso_languages)
df['spoken_languages_name'] = df['spoken_languages'].apply(extract_language_names)

df['languages_names_array'] = df['spoken_languages_name'].apply(lambda x: x if isinstance(x,list)else[])
aux4 = df[['languages_names_array']]  

In [18]:
aux4

,languages_names_array
0,[English]
1,"[English, Français]"
2,[English]
3,[English]
4,[English]
...,...
45461,[فارسی]
45462,[]
45463,[English]
45464,[]


Uno los Dataframe nuevos al original

In [38]:
df_final = pd.concat([aux, aux2, aux3, aux4],axis=1)


In [20]:
df_final.head(5)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,title,vote_average,vote_count,collection_id,collection_name,collection_poster_path,collection_backdrop_path,company_names_array,countries_names_array,languages_names_array
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,Toy Story,7.7,5415.0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,[Pixar Animation Studios],[United States of America],[English]
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,Jumanji,6.9,2413.0,NaN,None,None,None,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],"[English, Français]"
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,Grumpier Old Men,6.5,92.0,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg,"[Warner Bros., Lancaster Gate]",[United States of America],[English]
3,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,Waiting to Exhale,6.1,34.0,NaN,None,None,None,[Twentieth Century Fox Film Corporation],[United States of America],[English]
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,...,Father of the Bride Part II,5.7,173.0,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg,"[Sandollar Productions, Touchstone Pictures]",[United States of America],[English]


Se eliminan todas las columnas que no presentan utilidad para el posterior proceso creación del sistema de de recomendación.

In [21]:
df_final=df_final.drop(columns=["belongs_to_collection","production_companies","production_countries","spoken_languages","genres","tagline","collection_id","collection_poster_path"])

In [22]:
df_final.head(3)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,title,vote_average,vote_count,collection_name,collection_backdrop_path,company_names_array,countries_names_array,languages_names_array
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,Toy Story,7.7,5415.0,Toy Story Collection,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,[Pixar Animation Studios],[United States of America],[English]
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Jumanji,6.9,2413.0,None,None,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],"[English, Français]"
2,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg,"[Warner Bros., Lancaster Gate]",[United States of America],[English]


Se trasnforma en CSV para poder desarrollar una API luego.

In [25]:
df_final.to_csv(r'C:\Users\jessi\Desktop\Henry\PROYECTO FINAL INDIVIDUAL\movies_resultado.csv', index= False)

Se realiza el ETL del archivo credits.csv

In [26]:
base_dir = os.getcwd()

file_path = os.path.join(base_dir, 'C:\\Users\\jessi\\Desktop\\Henry\\PROYECTO FINAL INDIVIDUAL\\credits.csv')

df_credits = pd.read_csv(file_path)

Se separa en dos el dataframe

In [27]:
df_credit_cast = df_credits[['cast', 'id']].copy()

df_credit_crew = df_credits[['crew','id']].copy()

Se cambia el nombre a movie_id para facilitar la comprensión del código. Luego se parsean las columnas anidadas. El proceso puede demorar

In [28]:
df_credit_cast.rename(columns={'id':'movie_id'},inplace=True)

In [29]:
df_credit_crew.rename(columns={'id':'movie_id'},inplace=True)

In [30]:
 
def expand_columns(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else []) #Convierte el contenido de cada celda en la columna column de df de una cadena de texto que representa una lista de diccionarios a una lista de diccionarios Python, utilizando ast.literal_eval. Si el valor es nulo (pd.notnull(x)), se devuelve una lista vacía.
        df = df.explode(column)
        col_df = pd.json_normalize(df[column])
        col_df = col_df.add_prefix(f'{column}_')
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df)
    return df

columns_to_expand = ['cast']

df_credit_cast = expand_columns(df_credit_cast, columns_to_expand)

En total quedan dos datasets: df_credit_cast y df_credit_crew

In [ ]:
columns_to_expand = ['crew']
df_credit_crew = expand_columns(df_credit_crew, columns_to_expand)


Se eliminan columnas para poder crear un API luego.

In [31]:
df_credit_cast = df_credit_cast.drop(columns=["cast_cast_id","cast_credit_id","cast_id","cast_profile_path"])
#se eliminaron en total credit_id, order, profile_path y cast_credit_id


In [32]:
df_credit_crew = df_credit_crew.drop(columns=["crew_id"])


KeyError: "['crew_id'] not found in axis"

In [ ]:
df_credit_cast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564892 entries, 0 to 564891
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   movie_id        564892 non-null  int64  
 1   cast_character  562474 non-null  object 
 2   cast_gender     562474 non-null  float64
 3   cast_name       562474 non-null  object 
 4   cast_order      562474 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 21.5+ MB


In [ ]:
df_credit_crew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465085 entries, 0 to 465084
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   movie_id           465085 non-null  int64  
 1   crew_credit_id     464314 non-null  object 
 2   crew_department    464314 non-null  object 
 3   crew_gender        464314 non-null  float64
 4   crew_job           464314 non-null  object 
 5   crew_name          464314 non-null  object 
 6   crew_profile_path  95098 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 24.8+ MB


In [ ]:

output_file_path = r'C:\Users\jessi\Desktop\PI HENRY\Datasets\cast.parquet'
output_file_path1 = r'C:\Users\jessi\Desktop\PI Henry\Datasets\crew.parquet'
df_credit_cast.to_parquet(output_file_path)
df_credit_crew.to_parquet(output_file_path1)

Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0. Dataset: Movies

In [ ]:
df_final['revenue'].fillna(0)
df_final['budget'].fillna(0)

0        30000000
1        65000000
2               0
3        16000000
4               0
           ...   
45461           0
45462           0
45463           0
45464           0
45465           0
Name: budget, Length: 45466, dtype: object

Los valores nulos del campo release date deben eliminarse.
Dataset Movies


In [ ]:
df_final['release_date'].dropna()



0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45460    1991-05-13
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45379, dtype: object

Convertir en tipo fecha los datos de la columna Date_Release con correspondiente manejo de errores. Dataset Movies


In [ ]:
df_final['release_date'].fillna('1900-01-01')


0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45461    1900-01-01
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45466, dtype: object

In [ ]:
df_final['release_date']

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45461           NaN
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45466, dtype: object

In [ ]:
df_final['release_date'] = pd.to_datetime(df_final['release_date'], errors='coerce')

Generar una nueva columna llamada "release_year" en donde figurará el año de estreno de la película.Dataset Movies

In [ ]:
df_final['release_year']=df_final['release_date'].dt.year


Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0. Dataset Movies




In [ ]:
df_final['budget'] = pd.to_numeric(df_final['budget'], errors='coerce').astype('Int64')


Cada columna tiene sus NaN reemplazados por ceros.Dataset Movies


In [ ]:
df_final['revenue'].fillna(0, inplace=True)
df_final['budget'].fillna(0, inplace=True)

Se realiza la cuenta para generar el KPI y se reemplazan los "inf" y "NaN" con ceros. Dataset Movies


In [39]:
df_final['revenue'] = pd.to_numeric(df_final['revenue'], errors='coerce')
df_final['budget'] = pd.to_numeric(df_final['budget'], errors='coerce')

df_final['return'] = df_final['revenue'] / df_final['budget']

df_final['return'] = df_final['return'].replace(np.inf, 0).astype(float)

df_final['return'].fillna(0, inplace=True)

df_final.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,vote_average,vote_count,collection_id,collection_name,collection_poster_path,collection_backdrop_path,company_names_array,countries_names_array,languages_names_array,return
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,7.7,5415.0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,[Pixar Animation Studios],[United States of America],[English],12.451801
1,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,6.9,2413.0,NaN,None,None,None,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],"[English, Français]",4.043035
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,6.5,92.0,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg,"[Warner Bros., Lancaster Gate]",[United States of America],[English],0.000000
3,NaN,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,6.1,34.0,NaN,None,None,None,[Twentieth Century Fox Film Corporation],[United States of America],[English],5.090760
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,...,5.7,173.0,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg,"[Sandollar Productions, Touchstone Pictures]",[United States of America],[English],0.000000


In [40]:
df_final.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'collection_id', 'collection_name',
       'collection_poster_path', 'collection_backdrop_path',
       'company_names_array', 'countries_names_array', 'languages_names_array',
       'return'],
      dtype='object')